# Ames Housing Data - Modeling


### Contents:
- [Clean data in same method as training data](#Clean-data-in-same-method-as-training-data)
- [Matching the Columns](#Matching-the-Columns)
- [Create our Final CSV file](#Create-our-Final-CSV-file)
---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
import sklearn.linear_model as linear_model
import pickle
import csv

%matplotlib inline

## Load in trained model and scaler

In [2]:
with open ('../assets/ridge.pkl', 'rb') as f:
    ridge = pickle.load(f)
with open('../assets/scaler.pkl', 'rb') as f:
    ss = pickle.load(f)

## Load kaggle data

In [3]:
kaggle = pd.read_csv('../datasets/test.csv', index_col='Id')

## Clean data in same method as training data

In [4]:
with open('../assets/columns.pkl', 'rb') as f:
    columns = pickle.load(f)

In [5]:
kaggle.columns = [i.lower().replace(' ','_') for i in kaggle.columns]
kaggle.head()

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
Id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [6]:
# replace null values with more accurate values
kaggle.fillna(value={'mas_vnr_type': 'None','bsmt_qual': 'NA','bsmt_cond': 'NA','bsmt_exposure': 'NA','bsmtfin_type_1': 'NA',
                 'bsmtfin_type_2': 'NA','fireplace_qu': 'NA','garage_type': 'NA','garage_finish': 'NA','garage_qual': 'NA','garage_cond': 'NA',
                 'paved_drive': 'NA', 'mas_vnr_area': 0.0,'bsmtfin_sf_1': 0.0,'bsmtfin_sf_2': 0.0,'total_bsmt_sf': 0.0,'bsmt_full_bath': 0.0,
                 'bsmt_half_bath': 0.0, 'garage_cars': 0.0, 'garage_area': 0.0, 'fence': 0.0,'pool_qc':'NA'}, axis=0, inplace=True)

In [7]:
# dropping the columns that won't reflect the saleprice
kaggle.drop(['lot_frontage','pid','alley','pool_qc','fence','misc_feature','garage_yr_blt'], axis=1, inplace=True)

In [8]:
# set all the objects into a list to apply get_dummies
dummies_var = ['ms_zoning',
'street',
'lot_shape',
'land_contour',
'utilities',
'lot_config',
'land_slope',
'neighborhood',
'condition_1',
'condition_2',
'bldg_type',
'house_style', 
'roof_style',
'roof_matl',
'exterior_1st',
'exterior_2nd',
'mas_vnr_type',
'exter_qual',
'exter_cond',
'foundation',
'bsmt_qual',
'bsmt_cond',
'bsmt_exposure',
'bsmtfin_type_1',
'bsmtfin_type_2',
'heating',
'heating_qc',
'central_air',
'electrical',
'kitchen_qual',
'functional',
'fireplace_qu',
'garage_type',
'garage_finish',
'garage_qual',
'garage_cond',
'paved_drive',
'sale_type']

In [9]:
# get diummies on all object columns, and created a new dataframe of df_dummies
kaggle_dummies = pd.get_dummies(kaggle, columns = dummies_var)

In [10]:
# Combied exter_qual_TA and Gd and created a new column called exter_qual_TA_Gd
kaggle_dummies['exter_qual_TA_Gd'] = kaggle_dummies['exter_qual_TA'] + kaggle_dummies['exter_qual_Gd']
kaggle_dummies['kitchen_qual_TA_Gd'] = kaggle_dummies['kitchen_qual_TA'] + kaggle_dummies['kitchen_qual_Gd'] 

In [11]:
# Drop the original two columns after combining
kaggle_dummies.drop(['exter_qual_Gd','exter_qual_TA','kitchen_qual_TA','kitchen_qual_Gd'], axis=1, inplace=True)

In [12]:
with open('../assets/columns.pkl', 'rb') as f:
    columns = pickle.load(f)

In [13]:
kaggle_dummies.shape

(879, 266)

## Matching the Columns

In [14]:
# import my cleaned train data and set index as Id
df_dummies = pd.read_csv('../datasets/df_dummies.csv', index_col='Id')


In [15]:
# drop the saleprice on our train data
df_dummies.drop('saleprice', 1, inplace=True)

In [16]:
# finding the difference columns between our training and test data
missing_cols = set(columns).difference(kaggle_dummies.columns)

In [17]:
# making new columns that are missing in test data and fill it with 0
for col in missing_cols:
    kaggle_dummies[col] = 0

In [18]:
kaggle_dummies[columns].head()

,overall_qual,gr_liv_area,garage_area,total_bsmt_sf,1st_flr_sf,bsmt_qual_Ex,year_built,kitchen_qual_Ex,year_remod/add,full_bath,...,garage_finish_NA,garage_qual_NA,ms_zoning_RL,foundation_BrkTil,bsmt_qual_Gd,garage_type_BuiltIn,sale_type_WD,land_contour_HLS,neighborhood_OldTown,house_style_2Story
Id,,,,,,,,,,,,,,,,,,,,,
2658,6,1928,440,1020,908,0,1910,0,1950,2,...,0,0,0,0,0,0,1,0,1,1
2718,5,1967,580,1967,1967,0,1977,0,1977,2,...,0,0,1,0,1,0,1,0,0,0
2414,7,1496,426,654,664,0,2006,0,2006,2,...,0,0,1,0,1,0,0,0,0,1
1989,5,968,480,968,968,0,1923,0,2006,1,...,0,0,0,0,0,0,1,0,1,0
625,6,1394,514,1394,1394,0,1963,0,1963,1,...,0,0,1,0,1,0,1,0,0,0


In [19]:
kaggle_dummies.shape

(879, 266)

In [20]:
kaggle_dummies[columns].shape

(879, 69)

## Create our Final CSV file

In [21]:
kaggle_sc = ss.transform(kaggle_dummies[columns])

In [22]:
preds = ridge.predict(kaggle_sc)

In [23]:
preds.shape

(879, 1)

In [24]:
submission = pd.DataFrame(preds, index=kaggle.index, 
                          columns=['SalePrice'])

In [25]:
submission.sort_index(inplace=True)
submission.head()

,SalePrice
Id,
2,130967.238686
4,258806.315475
6,190332.129023
7,254233.352247
17,197026.843101


In [26]:
# use pandas .to_csv to save these predictions out
submission.to_csv('../datasets/second_attemp_ridge.csv')

In [27]:
!head ../datasets/first_attemp_ridge.csv

Id,SalePrice
2,128490.74592036601
4,264593.94841515645
6,189230.9758237811
7,240657.95918011857
17,193977.44801186892
18,379606.1502120754
22,194632.11758638136
27,116055.99333200656
31,99841.97265871533
